In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from datetime import datetime
from scipy import stats
# import hvplot.pandas

import requests
import json

# Import the API key
# from config import geoapify_key

In [4]:

from matplotlib import style
style.use('fivethirtyeight')

import datetime as dt

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy.ext.declarative import declarative_base

import sqlite3

Loading Data

In [19]:
# Read saved data: Registers and Local DB
QLD_IRA_db = pd.read_csv("Resources/QLD_IRA.csv", encoding="utf-8")

# Display sample data
QLD_IRA_db.head()



,IRAApparatusID,ApparatusType,Manufacturer,Model,SerialNumber,TubeDetails,LocationStorage,LocationUse,LocationRelocatingto,SN
0,I-AT46800,14 MeV neutron generator incorporating a H3 t...,Schlumberger Technology Corporation,PNG-GC,PNG-GC 67123-50503,No data,85 Duke Street ROMA QLD 4455,NaN,NaN,67123-50503
1,X61962,Linear accelerator apparatus,Schlumberger Technology Corporation,PNG-CB,8294-50388,No data,85 Duke Street ROMA QLD 4455,Throughout Queensland,NaN,8294-50388
2,X61963,Linear accelerator apparatus,Schlumberger Technology Corporation,PNG-CB,82596-43320,No data,85 Duke Street ROMA QLD 4455,Throughout Queensland,NaN,82596-43320
3,X61964,Linear accelerator apparatus,Schlumberger Technology Corporation,PNG-CB,8735-43534,No data,85 Duke Street ROMA QLD 4455,Throughout Queensland,NaN,8735-43534
4,X61965,Linear accelerator apparatus,Schlumberger Technology Corporation,PNG-CB,8864-50705,No data,85 Duke Street ROMA QLD 4455,Throughout Queensland,NaN,8864-50705


In [20]:
# Read saved data: Registers and Local DB
QLD_Reg_AR_db = pd.read_csv("Resources/QLD_Reg_AR.csv", encoding="utf-8")

# Display sample data
QLD_Reg_AR_db.head()



,ApparatusID,ApparatusManufacturer,Model,ApparatusSerialNumber,Purpose,SubstanceID,Radionuclide,Activity,MeasurementDate,UseLocation,StorageLocation,BusinessLine,ATA,Certificate,COC,InRMIS-NG,Status,FeedbackTIM,StatusShort,Evidence
0,I-AT46800,NaN,PNG-GC,67123-50503,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,RPE,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
1,X61962,NaN,PNG-CB,8294-50388,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,WCM,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
2,X61963,NaN,PNG-CB,8596-43320,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,WCM,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
3,X61964,NaN,PNG-CB,8735-43534,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,WCM,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
4,X61965,NaN,PNG-CB,8864-50705,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,WCM,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN


In [21]:
# Read saved data: Registers and Local DB
QLD_SRS_db = pd.read_csv("Resources/QLD_SRS.csv", encoding="utf-8")

# Display sample data
QLD_SRS_db.head()



,SRSSubstanceID,Radionuclide,Manufacturer,Model,SerialNumber,Physicalform,Chemicalform,Initialactivity,Initialactivitydate,DeviceID,DeviceType,DeviceManufacturer,DeviceModel,DeviceSerialNumber,LocationStorage,LocationUse,LocationRelocatingto
0,S48260,Am241/Be - Sealed radioactive substance,Eckert & Ziegler Isotope Products,AM1.N02,78-5887,Solid,Americium Oxide,1.48 GBQ,23-JUN-2009,D48258,Portable Soil density/moisture gauge,Troxler,3440,62988,"ROBINA, QLD 4226 - 3 Moorabbin Place",NaN,NaN
1,AT22030,Am241/Be - Sealed radioactive substance,Gammatron,AN-HP (DNS),G5224,Solid,Americium Oxide,592 GBq,12-DEC-2007,Unknown,Unknown,Unknown,Unknown,Unknown,ROMA - Store Southern Road,Throughout Queensland,NaN
2,S35491,Am241/Be - Sealed radioactive substance,Gammatron,NSR-F (AN-HP),G5380,NaN,NaN,592 GBq,28-APR-2014,Unknown,Unknown,Unknown,Unknown,Unknown,85 Duke Street ROMA QLD 4455,- Throughout Queensland,NaN
3,S35492,Am241/Be - Sealed radioactive substance,Gammatron,NSR-F (AN-HP),G5382,NaN,NaN,592 GBq,01-MAY-2014,Unknown,Unknown,Unknown,Unknown,Unknown,85 Duke Street ROMA QLD 4455,Throughout Queensland,NaN
4,S29663,Am241/Be - Sealed radioactive substance,Gammatron,NSR-F (AN-HP),G5251,Solid,Americium Oxide,592 GBq,02-MAR-2009,Unknown,Unknown,Unknown,Unknown,Unknown,85 Duke Street ROMA QLD 4455,Throughout Queensland,NaN


In [44]:
# Read saved data: Registers and Local DB
RMIS_db = pd.read_csv("Resources/RMIS_28Mar23.csv", encoding="utf-8")

# Display sample data
RMIS_db.head()



,Basin,Geounit,Country,FacilityNamewithFacilityID,BusinessLine,ModelID,SerialNo,CalibrationFrequencyinMonth,DateofLastCalibrationTest,CalibrationComplianceIndicator,...,ConfirmationInventoryFrequency,Dateoflastconfirmationinvtest,Dateofnextconfirmationinvtest,PhotoInventoryFrequency,DateofLastPhotoInvtest,DateofNextPhotoInvtest,ConfirmationPhotoComplianceIndicator,Located,PhysicalLocation,PAUControlSite
0,ASIA-ASIA,APG,Australia,Jandakot Base OFS Operating Base - 8509,GSS,XRD-BTX,8610/0085,NaN,(No Record),Non Compliance,...,3Month,2023-03-14,2023-06-14,12Month,2022-12-09,2023-12-09,Compliance,NaN,Shop,Jandakot Base
1,ASIA-ASIA,APG,Australia,Jandakot Base OFS Operating Base - 8509,GSS,XRF-SCOUT,12109147,NaN,(No Record),Non Compliance,...,3Month,2022-12-09,2023-03-09,12Month,2022-12-09,2023-12-09,Non Compliance,NaN,Tech Center,Jandakot Base
2,ASIA-ASIA,APG,Australia,Jandakot Base OFS Operating Base - 8509,RPE,NaN,SM|134968,12 Month,2022-07-26,Compliance,...,NaN,(Not Applicable),(Not Applicable),NaN,(Not Applicable),(Not Applicable),Ignore Case,OBS BAY,Shop,AUJAD-WLES
3,ASIA-ASIA,APG,Australia,Jandakot Base OFS Operating Base - 8509,RPE,NaN,SM|149029,12 Month,2022-03-14,Non Compliance,...,NaN,(Not Applicable),(Not Applicable),NaN,(Not Applicable),(Not Applicable),Ignore Case,OPCheck Bay,Shop,AUJAD-WLES
4,ASIA-ASIA,APG,Australia,Jandakot Base OFS Operating Base - 8509,RPE,NaN,SM|149035,12 Month,2023-03-15,Compliance,...,NaN,(Not Applicable),(Not Applicable),NaN,(Not Applicable),(Not Applicable),Ignore Case,OBS BAY,Shop,AUJAD-WLES


In [23]:
# Read saved data: Registers and Local DB
SA_db = pd.read_csv("Resources/SA.csv")

# Display sample data
SA_db.head()




,Description,Category,CertificateNumber,Isotope,Quantity,Form,Use,Manufacturer,Model,SerialNumber,ContainerModel,ContainerSerial,LocationonPremises,CurrentStatusLocation,BusinessLine,InRMISNG,Comments
0,Registration,Sealed Source,19337,Cs137,64 GBq,Sealed Solid,BORE HOLE PROBE,3M,GSR-J,3739,GCS-VC,672,"RA PIT 1, MOOMBA 5710",NaN,RPE,Yes,NaN
1,Registration,Sealed Source,20089,Am241,592 GBq,Sealed Solid,BORE HOLE PROBE,GAMMATRON,NSR-F,G5136,NCS-YC,5166,"Yellow Cell, MOOMBA 5710",NaN,RPE,Yes,NSR-FC in RMIS-NG
2,Registration,Sealed Source,22189,Cs137,62.53 GBq,Sealed Solid,BORE HOLE PROBE,AEA TECHNOLOGY,GSR-Z,A2262,NGCS-CB,11-Jun,"RA PIT, MOOMBA 5710",NaN,WCM,No,sent for RAN / PC-PC-NSSI
3,Registration,Sealed Source,22189,Am241,377.4 GBq,Sealed Solid,BORE HOLE PROBE,AEA TECHNOLOGY,NSR-U,N-392,NGCS-CB,11-Jun,"RA PIT, MOOMBA 5710",NaN,WCM,No,sent for RAN / PC-PC-NSSI
4,Registration,Sealed Source,22190,Cs137,60.59 GBq,Sealed Solid,BORE HOLE PROBE,AEA TECHNOLOGY,GSR-Z,7793B,NGCS-CB,13-May,"RA PIT, MOOMBA 5710",NaN,WCM,No,Junked - Disposed / PC-PC-NSSI


In [24]:
# Read saved data: Registers and Local DB
WA_Inventory_Amendments_db = pd.read_csv("Resources/WA_Inventory_Amendments.csv")

# Display sample data
WA_Inventory_Amendments_db.head()



,SourceN,Manufacturer,Model,SerialNumber,EquipmentUse,LocationonPremises,CurrentStatusLocation,BusinessLine,InRMISNG,Comments,Comm02,Comm03,Comm03.1
0,NaN,Schlumberger,PNG-CB,8955-51864,Logging,Jandakot Facility,NaN,WCM,Yes,Sent to Japan - RX Amendment sent 18-Oct-2022,NaN,NaN,NaN
1,22.0,Cs137,2.22 kBq,Sealed Solid,Calibration,-,EQ-602-2174,M2-136,Jandakot Facility,NaN,WCM,No,Sent to Malaysia - RS amednment sent 31-Oct-2022
2,25.0,Cs137,2.22 kBq,Sealed Solid,Calibration,-,EQ-602-2174,K1-273,Jandakot Facility,NaN,WCM,No,RS amendment sent - 7-Jun-22 (Sent to Malaysia)


In [25]:
# Read saved data: Registers and Local DB
WA_Inventory_RS_db = pd.read_csv("Resources/WA_Inventory_RS.csv")

# Display sample data
WA_Inventory_RS_db.head()



,SourceN,Isotope,Quantity,Form,PurposeorUse,Manufacturer,Model,SerialNumber,LocationonPremises,CurrentStatusLocation,BusinessLine,InRMISNG,Comments
0,1,H3,55.5 GBq,Sealed Gaseous,Gauge - logging,Schlumberger,PNG-AA,1013-51120,Balcatta Facility,NaN,RPE,Yes,NaN
1,2,Cs137,37 kBq,Sealed Solid,Calibration,Eckert & Ziegler,EQ-H343123,Q7-784,Jandakot,NaN,WCM,No,Missing in Qtrac\nInstalled in Detector# 97405...
2,3,Cs137,2.22kBq,Sealed Solid,Calibration,-,EQ-602-2174,P6-130,Jandakot,NaN,WCM,No,Amendment to add sent to WA council 22-Jul-22
3,4,Cs137,2.22kBq,Sealed Solid,Calibration,-,EQ-602-2174,P9-748,Jandakot,NaN,WCM,No,Amendment to add sent to WA council 22-Jul-22
4,5,Cs137,33.3kBq,Sealed Solid,Calibration,-,EQ-H619737,R5-597,Jandakot,NaN,WCM,Yes,Amendment to add sent to WA council 22-Jul-22


In [26]:
# Read saved data: Registers and Local DB
WA_Inventory_RX_db = pd.read_csv("Resources/WA_Inventory_RX.csv")

# Display sample data
WA_Inventory_RX_db.head()

,Model,SerialNumber,EquipmentUse,LocationonPremises,CurrentStatusLocation,BusinessLine,InRMISNG,Comments,Comments2
0,BTX,BTX-348,XRD,Jandakot Facility,NaN,WCM,No,GeoServices - registered but regulator require...,NaN
1,PNG-AA,1013-51120,Logging,Jandakot Facility,NaN,RPE,No,NaN,NaN
2,PNG-AA,1090-50652,Logging,Jandakot Facility,NaN,RPE,No,NaN,NaN
3,PNG-AA,1468-51086,Logging,Jandakot Facility,NaN,RPE,No,NaN,NaN
4,PNG-AA,1177-52252,Logging,Jandakot Facility,NaN,RPE,No,NaN,NaN


In [27]:
QLD_Reg_AR_db


,ApparatusID,ApparatusManufacturer,Model,ApparatusSerialNumber,Purpose,SubstanceID,Radionuclide,Activity,MeasurementDate,UseLocation,StorageLocation,BusinessLine,ATA,Certificate,COC,InRMIS-NG,Status,FeedbackTIM,StatusShort,Evidence
0,I-AT46800,NaN,PNG-GC,67123-50503,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,RPE,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
1,X61962,NaN,PNG-CB,8294-50388,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,WCM,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
2,X61963,NaN,PNG-CB,8596-43320,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,WCM,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
3,X61964,NaN,PNG-CB,8735-43534,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,WCM,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
4,X61965,NaN,PNG-CB,8864-50705,NaN,NaN,NaN,NaN,NaN,NaN,85 Duke Street,WCM,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,NaN,NaN,GSR-U,1006,NaN,S45289,Th,185 KBq,NaN,NaN,85 Duke Street,RPE,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN
314,NaN,NaN,GSR-U,1165,NaN,S45290,Th,185 KBq,NaN,NaN,85 Duke Street,RPE,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN
315,NaN,NaN,GSR-U,2006,NaN,S45291,Th,185 KBq,NaN,NaN,85 Duke Street,RPE,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN
316,NaN,NaN,GSR-U,2007,NaN,S45292,Th,185 KBq,NaN,NaN,85 Duke Street,RPE,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN


In [5]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/project.sqlite")

In [48]:
RMIS_db.to_sql('RMIS', con =engine)

In [30]:
QLD_IRA_db.to_sql('QLD_IRA', con = engine)

In [31]:
QLD_SRS_db.to_sql('QLD_SRS', con = engine)

In [32]:
QLD_Reg_AR_db.to_sql('QLD_Reg_AR', con = engine)

In [33]:
SA_db.to_sql('SA_Reg', con = engine)

In [34]:
WA_Inventory_RS_db.to_sql('WA_RS_Reg', con = engine)

In [35]:
WA_Inventory_RX_db.to_sql('WA_RX_Reg', con = engine)

In [6]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)
# Collect the names of tables within the database
inspector.get_table_names()


['QLD_IRA',
 'QLD_Reg_AR',
 'QLD_SRS',
 'RMIS',
 'SA_Reg',
 'WA_RS_Reg',
 'WA_RX_Reg']

In [51]:
# View all of the classes that automap found

# Using the inspector to print the column names within the 'measurement' table and its types

print ("---- QLD_IRA table information --------")

columns = inspector.get_columns('QLD_IRA')
for column in columns:
    print(column["name"], column["type"])



print ("---- QLD_Reg_AR table information --------")

columns = inspector.get_columns('QLD_Reg_AR')
for column in columns:
    print(column["name"], column["type"])


print ("\n ---- QLD_SRS table information --------")

columns = inspector.get_columns('QLD_SRS')
for column in columns:
    print(column["name"], column["type"])


print ("\n ---- RMIS table information --------")

columns = inspector.get_columns('RMIS')
for column in columns:
    print(column["name"], column["type"])


print ("\n ---- SA_Reg table information --------")

columns = inspector.get_columns('SA_Reg')
for column in columns:
    print(column["name"], column["type"])


print ("\n ---- WA_RS_Reg table information --------")

columns = inspector.get_columns('WA_RS_Reg')
for column in columns:
    print(column["name"], column["type"])


print ("\n ---- WA_RX_Reg table information --------")

columns = inspector.get_columns('WA_RX_Reg')
for column in columns:
    print(column["name"], column["type"])

---- QLD_IRA table information --------
index BIGINT
IRAApparatusID TEXT
ApparatusType TEXT
Manufacturer TEXT
Model TEXT
SerialNumber TEXT
TubeDetails TEXT
LocationStorage TEXT
LocationUse TEXT
LocationRelocatingto TEXT
SN TEXT
---- QLD_Reg_AR table information --------
index BIGINT
ApparatusID TEXT
ApparatusManufacturer FLOAT
Model TEXT
ApparatusSerialNumber TEXT
Purpose FLOAT
SubstanceID TEXT
Radionuclide TEXT
Activity TEXT
MeasurementDate FLOAT
UseLocation FLOAT
StorageLocation TEXT
BusinessLine TEXT
ATA FLOAT
Certificate FLOAT
COC FLOAT
InRMIS-NG TEXT
Status TEXT
FeedbackTIM TEXT
StatusShort TEXT
Evidence TEXT

 ---- QLD_SRS table information --------
index BIGINT
SRSSubstanceID TEXT
Radionuclide TEXT
Manufacturer TEXT
Model TEXT
SerialNumber TEXT
Physicalform TEXT
Chemicalform TEXT
Initialactivity TEXT
Initialactivitydate TEXT
DeviceID TEXT
DeviceType TEXT
DeviceManufacturer TEXT
DeviceModel TEXT
DeviceSerialNumber TEXT
LocationStorage TEXT
LocationUse TEXT
LocationRelocatingto TE

['QLD_IRA',
 'QLD_Reg_AR',
 'QLD_SRS',
 'RMIS',
 'SA_Reg',
 'WA_RS_Reg',
 'WA_RX_Reg']


In [7]:
# reflect an existing database into a new model

Base = automap_base()
Base.prepare(autoload_with=engine, reflect=True)


# reflect the tables
Base.classes.keys()

['QLD_IRA',
 'QLD_Reg_AR',
 'QLD_SRS',
 'RMIS',
 'SA_Reg',
 'WA_RS_Reg',
 'WA_RX_Reg']

In [8]:
# Save references to each table

# Map RMIS class
RMIS = Base.classes.RMIS

# Map QLD_IRA class
QLD_IRA = Base.classes.QLD_IRA

# Map QLD_Reg_AR class
QLD_Reg_AR = Base.classes.QLD_Reg_AR

# Map QLD_SRS class
QLD_SRS_AR = Base.classes.QLD_SRS

# Map SA_Reg class
SA_Reg = Base.classes.SA_Reg

# Map WA_RS_Reg class
WA_RS_Reg = Base.classes.WA_RS_Reg

# Map WA_RX_Reg class
WA_RX_Reg = Base.classes.WA_RX_Reg

In [9]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [55]:

print ("\n ---- RMIS table information --------")

columns = inspector.get_columns('RMIS')
for column in columns:
    print(column["name"], column["type"])


 ---- RMIS table information --------
index BIGINT
Basin TEXT
Geounit TEXT
Country TEXT
FacilityNamewithFacilityID TEXT
BusinessLine TEXT
ModelID TEXT
SerialNo TEXT
CalibrationFrequencyinMonth TEXT
DateofLastCalibrationTest TEXT
CalibrationComplianceIndicator TEXT
LeakTestFrequencyinMonth TEXT
DateofLastLeakTest TEXT
LeakTestComplianceIndicator TEXT
LastLeakTestResultRecorded TEXT
LeakTestResultComplianceIndicator TEXT
ConfirmationInventoryFrequency TEXT
Dateoflastconfirmationinvtest TEXT
Dateofnextconfirmationinvtest TEXT
PhotoInventoryFrequency TEXT
DateofLastPhotoInvtest TEXT
DateofNextPhotoInvtest TEXT
ConfirmationPhotoComplianceIndicator TEXT
Located TEXT
PhysicalLocation TEXT
PAUControlSite TEXT


In [10]:
# Photo inventory dates dates


print ("\n----- Total dates------\n")
totald = session.query(func.count(RMIS.DateofLastPhotoInvtest)).\
    filter(RMIS.PhotoInventoryFrequency == '12Month').\
    order_by(RMIS.DateofLastPhotoInvtest).all()
print (totald)

# Earliest Date

print ("\n----- Earliest Date------\n")
date = session.query(RMIS.DateofLastPhotoInvtest).order_by(RMIS.DateofLastPhotoInvtest).\
    filter(RMIS.PhotoInventoryFrequency == '12Month').\
    order_by(RMIS.DateofLastPhotoInvtest).first()
print (date)


# Latest Date

print ("\n----- Latest Date-------\n")
date = session.query(RMIS.DateofLastPhotoInvtest).order_by(RMIS.DateofLastPhotoInvtest.desc()).\
    filter(RMIS.PhotoInventoryFrequency == '12Month').\
    order_by(RMIS.DateofLastPhotoInvtest).first()
print (date)


----- Total dates------

[(833,)]

----- Earliest Date------

('2021-08-10',)

----- Latest Date-------

('2023-09-27',)


In [11]:
print ("\n----- Records in a location------\n")
totald = session.query(RMIS.BusinessLine,func.count(RMIS.FacilityNamewithFacilityID)).\
    filter(RMIS.FacilityNamewithFacilityID == 'Jandakot Base OFS Operating Base - 8509').\
    group_by(RMIS.BusinessLine).\
    all()
print (totald)


----- Records in a location------

[('GSS', 2), ('RPE', 3), ('RPI', 5), ('RPS', 4), ('WCM', 124)]


In [12]:
print ("\n----- Records in a location------\n")
totald = session.query(RMIS.FacilityNamewithFacilityID, RMIS.BusinessLine,func.count(RMIS.FacilityNamewithFacilityID)).\
     group_by(RMIS.FacilityNamewithFacilityID).\
     order_by(RMIS.BusinessLine).\
    all()
print (totald)


----- Records in a location------

[('JOP - Cikarang Integrated Base (CIB) OFS Base - 7559', 'GSS', 254), ('Jandakot Base OFS Operating Base - 8509', 'GSS', 138), ('Chengdu OFS Base - 560', 'RPE', 61), ('JOP - Elnusa Open Yard Cilandak - 2763', 'RPE', 4), ('KAL - Manggar OFS Base - 255', 'RPE', 213), ('Korla Integrated Base - 1282', 'RPE', 35), ('LinGang Integrated Base - 13061', 'RPE', 88), ('Moomba WL Operating Base - 1603', 'RPE', 20), ('Nagaoka OFS Base - 1644', 'RPE', 32), ('New Plymouth OFS Base Integrated Base - 1696', 'RPE', 24), ('Newburn Base - 1839', 'RPE', 63), ('PNG OFS Base Integrated Base - 8167', 'RPE', 16), ('Roma WL Operating Base - 2074', 'RPE', 52), ('SUM - Duri OFS Base - 789', 'RPE', 76), ('SUM - Jambi WL Base - 1205', 'RPE', 6), ('SUM - Prabumulih OFS Base - 1943', 'RPE', 13), ('Shekou Oilfield Services Base - 2171', 'RPE', 45), ('Taiwan Miaoli base - 1548', 'RPE', 9), ('Toowoomba OFS Operating Base - 10493', 'RPS', 7), ('Huizhou Radioactive Warehouse - 18099', 

In [44]:
# Calibration compliance

print ("\n----- Calibration compliance------\n")
CalComp = session.query(RMIS.FacilityNamewithFacilityID, RMIS.BusinessLine, RMIS.CalibrationComplianceIndicator, func.count(RMIS.CalibrationComplianceIndicator)).\
    group_by(RMIS.FacilityNamewithFacilityID).\
    group_by(RMIS.BusinessLine).\
    group_by(RMIS.CalibrationComplianceIndicator).\
    all()
print (CalComp)


----- Calibration compliance------

[('Chengdu OFS Base - 560', 'RPE', 'Compliance', 2), ('Chengdu OFS Base - 560', 'RPE', 'Non Compliance', 39), ('Chengdu OFS Base - 560', 'RPS', 'Compliance', 2), ('Chengdu OFS Base - 560', 'RPS', 'Critical', 1), ('Chengdu OFS Base - 560', 'RPS', 'Non Compliance', 12), ('Chengdu OFS Base - 560', 'WCM', 'Compliance', 2), ('Chengdu OFS Base - 560', 'WCM', 'Non Compliance', 3), ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Action Needed', 1), ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Compliance', 15), ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Critical', 2), ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Non Compliance', 81), ('JOP - Cikarang Integrated Base (CIB) OFS Base - 7559', 'GSS', 'Non Compliance', 1), ('JOP - Cikarang Integrated Base (CIB) OFS Base - 7559', 'RPE', 'Compliance', 17), ('JOP - Cikarang Integrated Base (CIB) OFS Base - 7559', 'RPE', 'Non Compliance', 126), ('JOP - Cikarang Integrated Base (CIB) OFS Base - 7559

In [38]:
CalComp


[('Chengdu OFS Base - 560', 'RPE', 'Compliance', 2),
 ('Chengdu OFS Base - 560', 'RPE', 'Non Compliance', 39),
 ('Chengdu OFS Base - 560', 'RPS', 'Compliance', 2),
 ('Chengdu OFS Base - 560', 'RPS', 'Critical', 1),
 ('Chengdu OFS Base - 560', 'RPS', 'Non Compliance', 12),
 ('Chengdu OFS Base - 560', 'WCM', 'Compliance', 2),
 ('Chengdu OFS Base - 560', 'WCM', 'Non Compliance', 3),
 ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Action Needed', 1),
 ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Compliance', 15),
 ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Critical', 2),
 ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Non Compliance', 81),
 ('JOP - Cikarang Integrated Base (CIB) OFS Base - 7559', 'GSS', 'Non Compliance', 1),
 ('JOP - Cikarang Integrated Base (CIB) OFS Base - 7559', 'RPE', 'Compliance', 17),
 ('JOP - Cikarang Integrated Base (CIB) OFS Base - 7559', 'RPE', 'Non Compliance', 126),
 ('JOP - Cikarang Integrated Base (CIB) OFS Base - 7559', 'RPI', 'Non Complian

In [46]:
#Leak Test compliance

print ("\n----- Leak Testcompliance------\n")
LeakTestComp = session.query(RMIS.FacilityNamewithFacilityID, RMIS.BusinessLine, RMIS.LeakTestComplianceIndicator, func.count(RMIS.LeakTestComplianceIndicator)).\
    group_by(RMIS.FacilityNamewithFacilityID).\
    group_by(RMIS.BusinessLine).\
    group_by(RMIS.LeakTestComplianceIndicator).\
    all()
print (LeakTestComp)


----- Leak Testcompliance------

[('Chengdu OFS Base - 560', 'RPE', 'Compliance', 23), ('Chengdu OFS Base - 560', 'RPE', 'Non Compliance', 2), ('Chengdu OFS Base - 560', 'RPE', 'Not Applicable', 16), ('Chengdu OFS Base - 560', 'RPS', 'Compliance', 10), ('Chengdu OFS Base - 560', 'RPS', 'Critical', 2), ('Chengdu OFS Base - 560', 'RPS', 'Non Compliance', 3), ('Chengdu OFS Base - 560', 'WCM', 'Non Compliance', 2), ('Chengdu OFS Base - 560', 'WCM', 'Not Applicable', 3), ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Compliance', 21), ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Critical', 1), ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Non Compliance', 22), ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Not Applicable', 55), ('JOP - Cikarang Integrated Base (CIB) OFS Base - 7559', 'GSS', 'Compliance', 1), ('JOP - Cikarang Integrated Base (CIB) OFS Base - 7559', 'RPE', 'Action Needed', 5), ('JOP - Cikarang Integrated Base (CIB) OFS Base - 7559', 'RPE', 'Compliance', 55), (

In [49]:
# Photo compliance

print ("\n----- Photo compliance------\n")
PhotoComp = session.query(RMIS.FacilityNamewithFacilityID, RMIS.BusinessLine, RMIS.ConfirmationPhotoComplianceIndicator, func.count(RMIS.ConfirmationPhotoComplianceIndicator)).\
    group_by(RMIS.FacilityNamewithFacilityID).\
    group_by(RMIS.BusinessLine).\
    group_by(RMIS.ConfirmationPhotoComplianceIndicator).\
    all()
print (PhotoComp)


----- Photo compliance------

[('Chengdu OFS Base - 560', 'RPE', 'Action Needed', 9), ('Chengdu OFS Base - 560', 'RPE', 'Compliance', 22), ('Chengdu OFS Base - 560', 'RPE', 'Critical', 2), ('Chengdu OFS Base - 560', 'RPE', 'Ignore Case', 2), ('Chengdu OFS Base - 560', 'RPE', 'Not Applicable', 6), ('Chengdu OFS Base - 560', 'RPS', 'Critical', 11), ('Chengdu OFS Base - 560', 'RPS', 'Ignore Case', 3), ('Chengdu OFS Base - 560', 'RPS', 'Non Compliance', 1), ('Chengdu OFS Base - 560', 'WCM', 'Compliance', 2), ('Chengdu OFS Base - 560', 'WCM', 'Ignore Case', 2), ('Chengdu OFS Base - 560', 'WCM', 'Non Compliance', 1), ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Action Needed', 10), ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Compliance', 49), ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Critical', 4), ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Ignore Case', 22), ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Non Compliance', 3), ('Huizhou Radioactive Warehouse - 18

In [ ]:
# Total dates

print ("\n----- Total dates------\n")
totald = session.query(func.count(RMIS.Geounit)).all()
print (totald)

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# The most recent day as per the query above is 2017-08-23

# Calculate the date one year from the last date in data set.

#One year is going be assumed as 365 days as the last day was in 2017 which had 365 days (The year 2017 had 365 days. The year 2016 had 366 days.)

query_date = dt.date(2023, 4, 9) - dt.timedelta(days=30)
print("Query Date: ", query_date)


# Perform a query to retrieve the data and precipitation scores


prcp_data = session.query(Measurement.date, Measurement.prcp).\
     filter(Measurement.date >= query_date).\
     order_by(Measurement.date).all()


In [69]:
print ("\n----- Photo compliance------\n")
PhotoComp = session.query(RMIS.Geounit, RMIS.Country, RMIS.FacilityNamewithFacilityID, RMIS.BusinessLine, RMIS.ConfirmationPhotoComplianceIndicator, func.count(RMIS.ConfirmationPhotoComplianceIndicator)).\
    group_by(RMIS.FacilityNamewithFacilityID).\
    group_by(RMIS.BusinessLine).\
    group_by(RMIS.ConfirmationPhotoComplianceIndicator).\
    all()
print (PhotoComp)



----- Photo compliance------

[('CHG', 'China', 'Chengdu OFS Base - 560', 'RPE', 'Action Needed', 9), ('CHG', 'China', 'Chengdu OFS Base - 560', 'RPE', 'Compliance', 22), ('CHG', 'China', 'Chengdu OFS Base - 560', 'RPE', 'Critical', 2), ('CHG', 'China', 'Chengdu OFS Base - 560', 'RPE', 'Ignore Case', 2), ('CHG', 'China', 'Chengdu OFS Base - 560', 'RPE', 'Not Applicable', 6), ('CHG', 'China', 'Chengdu OFS Base - 560', 'RPS', 'Critical', 11), ('CHG', 'China', 'Chengdu OFS Base - 560', 'RPS', 'Ignore Case', 3), ('CHG', 'China', 'Chengdu OFS Base - 560', 'RPS', 'Non Compliance', 1), ('CHG', 'China', 'Chengdu OFS Base - 560', 'WCM', 'Compliance', 2), ('CHG', 'China', 'Chengdu OFS Base - 560', 'WCM', 'Ignore Case', 2), ('CHG', 'China', 'Chengdu OFS Base - 560', 'WCM', 'Non Compliance', 1), ('CHG', 'China', 'Huizhou Radioactive Warehouse - 18099', 'WCM', 'Action Needed', 10), ('CHG', 'China', 'Huizhou Radioactive Warehouse - 18099', 'WCM', 'Compliance', 49), ('CHG', 'China', 'Huizhou Radioac

In [51]:
PhotoComp


[('Chengdu OFS Base - 560', 'RPE', 'Action Needed', 9),
 ('Chengdu OFS Base - 560', 'RPE', 'Compliance', 22),
 ('Chengdu OFS Base - 560', 'RPE', 'Critical', 2),
 ('Chengdu OFS Base - 560', 'RPE', 'Ignore Case', 2),
 ('Chengdu OFS Base - 560', 'RPE', 'Not Applicable', 6),
 ('Chengdu OFS Base - 560', 'RPS', 'Critical', 11),
 ('Chengdu OFS Base - 560', 'RPS', 'Ignore Case', 3),
 ('Chengdu OFS Base - 560', 'RPS', 'Non Compliance', 1),
 ('Chengdu OFS Base - 560', 'WCM', 'Compliance', 2),
 ('Chengdu OFS Base - 560', 'WCM', 'Ignore Case', 2),
 ('Chengdu OFS Base - 560', 'WCM', 'Non Compliance', 1),
 ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Action Needed', 10),
 ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Compliance', 49),
 ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Critical', 4),
 ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Ignore Case', 22),
 ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'Non Compliance', 3),
 ('Huizhou Radioactive Warehouse - 18099', 'WCM', 'N

In [53]:
PhotoComp_df = pd.DataFrame(PhotoComp, columns=['FacilityNamewithFacilityID', 'BusinessLine', 'ConfirmationPhotoComplianceIndicator','Count'])

In [54]:
PhotoComp_df

,FacilityNamewithFacilityID,BusinessLine,ConfirmationPhotoComplianceIndicator,Count
0,Chengdu OFS Base - 560,RPE,Action Needed,9
1,Chengdu OFS Base - 560,RPE,Compliance,22
2,Chengdu OFS Base - 560,RPE,Critical,2
3,Chengdu OFS Base - 560,RPE,Ignore Case,2
4,Chengdu OFS Base - 560,RPE,Not Applicable,6
...,...,...,...,...
125,Taiwan Miaoli base - 1548,RPE,Critical,2
126,Taiwan Miaoli base - 1548,RPE,Ignore Case,2
127,Toowoomba OFS Operating Base - 10493,RPS,Compliance,4
128,Toowoomba OFS Operating Base - 10493,RPS,Ignore Case,3


In [52]:
 PhotoComp_data_out = []

In [56]:
len(PhotoComp)

130

In [60]:
PhotoComp[0].FacilityNamewithFacilityID

'Chengdu OFS Base - 560'

In [61]:
PhotoComp[0].BusinessLine

'RPE'

In [62]:
PhotoComp[0].ConfirmationPhotoComplianceIndicator

'Action Needed'

In [65]:
PhotoComp[0][3]

9

In [67]:
    for row in PhotoComp:
        PhotoComp_dict = {}
        PhotoComp_dict["FacilityNamewithFacilityID"]=row.FacilityNamewithFacilityID
        PhotoComp_dict["BusinessLine"]=row.BusinessLine
        PhotoComp_dict["ConfirmationPhotoComplianceIndicator"]=row.ConfirmationPhotoComplianceIndicator
        PhotoComp_dict["Count"]=row[3]

        PhotoComp_data_out.append(PhotoComp_dict)

In [68]:
PhotoComp_data_out

[{'FacilityNamewithFacilityID': 'Chengdu OFS Base - 560',
  'BusinessLine': 'RPE',
  'ConfirmationPhotoComplianceIndicator': 'Action Needed',
  'Count': 9},
 {'FacilityNamewithFacilityID': 'Chengdu OFS Base - 560',
  'BusinessLine': 'RPE',
  'ConfirmationPhotoComplianceIndicator': 'Compliance',
  'Count': 22},
 {'FacilityNamewithFacilityID': 'Chengdu OFS Base - 560',
  'BusinessLine': 'RPE',
  'ConfirmationPhotoComplianceIndicator': 'Critical',
  'Count': 2},
 {'FacilityNamewithFacilityID': 'Chengdu OFS Base - 560',
  'BusinessLine': 'RPE',
  'ConfirmationPhotoComplianceIndicator': 'Ignore Case',
  'Count': 2},
 {'FacilityNamewithFacilityID': 'Chengdu OFS Base - 560',
  'BusinessLine': 'RPE',
  'ConfirmationPhotoComplianceIndicator': 'Not Applicable',
  'Count': 6},
 {'FacilityNamewithFacilityID': 'Chengdu OFS Base - 560',
  'BusinessLine': 'RPS',
  'ConfirmationPhotoComplianceIndicator': 'Critical',
  'Count': 11},
 {'FacilityNamewithFacilityID': 'Chengdu OFS Base - 560',
  'BusinessL

In [43]:
# Close Session
session.close()

In [70]:
PhotoComp

[('CHG', 'China', 'Chengdu OFS Base - 560', 'RPE', 'Action Needed', 9),
 ('CHG', 'China', 'Chengdu OFS Base - 560', 'RPE', 'Compliance', 22),
 ('CHG', 'China', 'Chengdu OFS Base - 560', 'RPE', 'Critical', 2),
 ('CHG', 'China', 'Chengdu OFS Base - 560', 'RPE', 'Ignore Case', 2),
 ('CHG', 'China', 'Chengdu OFS Base - 560', 'RPE', 'Not Applicable', 6),
 ('CHG', 'China', 'Chengdu OFS Base - 560', 'RPS', 'Critical', 11),
 ('CHG', 'China', 'Chengdu OFS Base - 560', 'RPS', 'Ignore Case', 3),
 ('CHG', 'China', 'Chengdu OFS Base - 560', 'RPS', 'Non Compliance', 1),
 ('CHG', 'China', 'Chengdu OFS Base - 560', 'WCM', 'Compliance', 2),
 ('CHG', 'China', 'Chengdu OFS Base - 560', 'WCM', 'Ignore Case', 2),
 ('CHG', 'China', 'Chengdu OFS Base - 560', 'WCM', 'Non Compliance', 1),
 ('CHG', 'China', 'Huizhou Radioactive Warehouse - 18099', 'WCM', 'Action Needed', 10),
 ('CHG', 'China', 'Huizhou Radioactive Warehouse - 18099', 'WCM', 'Compliance', 49),
 ('CHG', 'China', 'Huizhou Radioactive Warehouse - 1

In [71]:
PhotoComp_df = pd.DataFrame(PhotoComp, columns=['Geounit', 'Country', 'FacilityNamewithFacilityID', 'BusinessLine', 'ConfirmationPhotoComplianceIndicator','Count'])

In [81]:
bases= PhotoComp_df['FacilityNamewithFacilityID'].unique()

In [82]:
bases

array(['Chengdu OFS Base - 560', 'Huizhou Radioactive Warehouse - 18099',
       'JOP - Cikarang Integrated Base (CIB) OFS Base - 7559',
       'JOP - Elnusa Open Yard Cilandak - 2763',
       'Jandakot Base OFS Operating Base - 8509',
       'KAL - Manggar OFS Base - 255', 'Korla Integrated Base - 1282',
       'LinGang Integrated Base - 13061',
       'Moomba WL Operating Base - 1603', 'Nagaoka OFS Base - 1644',
       'New Plymouth OFS Base Integrated Base - 1696',
       'Newburn Base - 1839', 'PNG OFS Base Integrated Base - 8167',
       'Roma Smith Pathfinder Operating Base - 2073',
       'Roma WL Operating Base - 2074', 'SUM - Duri OFS Base - 789',
       'SUM - Jambi WL Base - 1205', 'SUM - Prabumulih OFS Base - 1943',
       'Shekou Oilfield Services Base - 2171',
       'Taiwan Miaoli base - 1548',
       'Toowoomba OFS Operating Base - 10493',
       'XiQing Product Center - 2353'], dtype=object)

In [84]:
countries = PhotoComp_df['Country'].unique()

In [89]:
countries

array(['China', 'Indonesia', 'Australia', 'Japan', 'New Zealand',
       'Papua New Guinea', 'Taiwan, Province Of China'], dtype=object)

In [87]:
cities = ['China Chengdu OFS Base - 560', 'China Huizhou Radioactive Warehouse - 18099', 'Indonesia JOP - Cikarang Integrated Base CIB) OFS Base - 7559', 'Indonesia JOP - Elnusa Open Yard Cilandak - 2763','Australia Jandakot Base OFS Operating Base - 8509','Indonesia KAL - Manggar OFS Base - 255', 'China Korla Integrated Base - 1282', 'China LinGang Integrated Base - 13061', 'Australia Moomba WL Operating Base - 1603', 'Japan Nagaoka OFS Base - 1644', 'New Zealand New Plymouth OFS Base Integrated Base - 1696', 'Australia Newburn Base - 1839', 'Papua New Guinea', 'PNG OFS Base Integrated Base - 8167', 'Australia Roma Smith Pathfinder Operating Base - 2073', 'Australia Roma WL Operating Base - 2074', 'Indonesia SUM - Duri OFS Base - 789',  'Indonesia SUM - Jambi WL Base - 1205', 'Indonesia SUM - Prabumulih OFS Base - 1943', 'China Shekou Oilfield Services Base - 2171', 'Taiwan, Province Of China Taiwan Miaoli base - 1548', 'Australia Toowoomba OFS Operating Base - 10493','China XiQing Product Center - 2353']

In [ ]:
# Close Session
session.close()